## Test of cbpro library

In [12]:
import pandas as pd
from datetime import date
import cbpro
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential

In [1]:
with open('CoinBaseAPIKey.key', 'r') as f:
    contents = f.readlines()
cb_key = contents[0].split(':')[-1].strip()
cb_secret = contents[1].split(':')[-1].strip()

In [3]:
public_client = cbpro.PublicClient()

BTC_USDT = public_client.get_product_historic_rates('BTC-USDT', '2023-01-01', '2023-06-01', 86400)
ETH_USDT = public_client.get_product_historic_rates('ETH-USDT', '2023-01-01', '2023-06-01', 86400)
XRP_USDT = public_client.get_product_historic_rates('XRP-USDT', '2023-01-01', '2023-06-01', 86400)

In [5]:
df_hist_data= pd.DataFrame(BTC_USDT)
df_hist_data.columns = [ 'date', 'BTC_low', 'BTC_high', 'BTC_open', 'BTC_close', 'BTC_volume' ]
df_hist_data['date']= df_hist_data['date'].apply(lambda x: date.fromtimestamp(x))
coin_res = [ETH_USDT]
coin_names = ['ETH']
for coin, name in zip (coin_res, coin_names):
    tmp = pd.DataFrame(coin)
    tmp.columns = [ 'date', 'low', 'high', 'open', 'close', 'volume' ]
df_hist_data.head()

,date,BTC_low,BTC_high,BTC_open,BTC_close,BTC_volume
0,2023-05-31,26611.84,27351.09,27220.14,26822.11,567.223972
1,2023-05-30,26841.45,27838.36,27689.23,27215.28,991.007433
2,2023-05-29,27552.65,28040.51,27736.03,27695.30,502.509372
3,2023-05-28,27530.82,28451.62,28081.19,27736.96,673.158599
4,2023-05-27,26775.89,28271.94,26859.82,28082.01,712.806237


In [13]:
scaler  = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(np.array(df_hist_data['BTC_close'].values).reshape(-1,1) )

array([[0.73702816],
       [0.76542695],
       [0.80009896],
       [0.80310807],
       [0.82803115],
       [0.7396465 ],
       [0.72862127],
       [0.71229288],
       [0.70124742],
       [0.76584806],
       [0.73893431],
       [0.73083657],
       [0.75752134],
       [0.74156566],
       [0.73707077],
       [0.77911966],
       [0.75241394],
       [0.76163847],
       [0.74409589],
       [0.73367017],
       [0.73476518],
       [0.7477847 ],
       [0.79282175],
       [0.79573047],
       [0.79803678],
       [0.8532287 ],
       [0.88355677],
       [0.9312779 ],
       [0.8832599 ],
       [0.89641304],
       [0.87040651],
       [0.82705676],
       [0.90908369],
       [0.91101729],
       [0.91704852],
       [0.92948514],
       [0.85189099],
       [0.8443747 ],
       [0.7866894 ],
       [0.79210089],
       [0.80901146],
       [0.76887451],
       [0.83997154],
       [0.87979645],
       [0.9938467 ],
       [0.92574144],
       [0.98854499],
       [0.988